In [ ]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from humanize import number

# Functions and Intializations

In [ ]:
#Intializing Global Lists
AllPersons = []
WorksAt = []
Movies = []
MovieProdComp = []
Nomination = []
AcademyAwards = []
AcademyAwardsHost = []


In [ ]:
#Intializing the Roles table
Roles =  {
  "Directed by" : "Director",
  "Produced by" : "Producer",
  "Written by" : "Writter",
  "Starring" : "Actor",
  "Screenplay by" : "Screenwriter",
  "Cinematography" : "Cinematographer",
  "Music by": "Composer"

}

## Date Function

In [ ]:
def ExtractDate(text):
  #Cleaning the text first

  text = re.sub(r'\(.*?\)', '', text)
  text = re.sub(r'\[.*?\]', '', text)

  # Putting the different patterns I observed
  DatePatterns = [
      r'\d{4}-\d{1,2}-\d{1,2}',
      r'\w+ \d{1,2}, \d{4}',
      r'\d{1,2} \w+ \d{4}',
      r'\w+ \d{4}',
      r'\d{4}'

  ]

  # Looking if the text passed matches any of the patterns I observed
  for date in DatePatterns:
    Dt = re.search(date, text)
    if Dt:
      try:
        Date = pd.to_datetime(Dt.group(), errors = 'coerce')
        if pd.notnull(Date):
          return Date.strftime('%Y-%m-%d')
      except:
        continue

  return None

## Extracting All Links Function

In [ ]:
def AcquiringLinks(soup):
  Table = soup.find("table" , {"class" : "wikitable"})
  TableBody = Table.find("tbody")
  Links = set() # I used a set to make it unique as in the wikitable the same link for the same movie/person could be repeated alot
  MainUrl = "https://en.wikipedia.org"
  for tr in TableBody.find_all("tr"):
    for td in tr.find_all("td"):
      for ul in td.find_all("ul"):
        for li in ul.find_all("li"):
          for a in li.find_all("a"):
            href = a.get("href")
            if href:
              if href.startswith("http"):
                Links.add(href)
              else:
                Links.add(MainUrl + a["href"])
  return Links





## Checking if Link is a Movie Function

In [ ]:
def IsLinkMovie(link):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
        response = requests.get(link, headers=headers, timeout=6)
        soup = BeautifulSoup(response.content, "html.parser")

        # Check if '(film)' is in the page title
        page_title = soup.title.string if soup.title else ""
        if "(film)" in page_title.lower():
            return True

        # Check for infobox
        infobox = soup.find("table", class_="infobox vevent")
        if not infobox:
            return False

        movie_identifiers = [
            "Directed by", "Produced by", "Starring",
            "Running time", "Release date", "Production company",
            "Distributed by", "Cinematography", "Edited by"
        ]

        for th in infobox.find_all("th"):
            label = th.get_text(strip=True)
            if label in movie_identifiers:
                return True

        return False

    except Exception as e:
        print(f"[IsLinkMovie ERROR] {link} → {e}")
        return False


In [ ]:
print(IsLinkMovie("https://en.wikipedia.org/wiki/The_Devil_Dancer"))

True


## Checking if Link is a Person Function

In [ ]:
def IsLinkPerson(link):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
        response = requests.get(link, headers=headers, timeout=6)
        soup = BeautifulSoup(response.content, "html.parser")

        infobox = soup.find("table", class_=lambda c: c and (
            "infobox biography vcard" in c or
            "infobox vcard plainlist" in c
        ))
        if not infobox:
            #print(f"[IsLinkPerson] No infobox found for: {link}")
            return False

        person_identifiers = [
            "Born", "Occupation", "Years active", "Spouse(s)", "Children",
            "Nationality", "Education", "Alma mater", "Known for"
        ]

        for th in infobox.find_all("th"):
            label = th.get_text(strip=True)
            if label in person_identifiers:
                #print(f"[IsLinkPerson] Confirmed by label '{label}' → {link}")
                return True

        #print(f"[IsLinkPerson] Not a person (no identifiers found): {link}")
        return False

    except Exception as e:
        print(f"[IsLinkPerson ERROR] {link} → {e}")
        return False


In [ ]:
print(IsLinkPerson("https://en.wikipedia.org/wiki/Annette_Bening"))

True


## Checking if Link is a Produciton Com Function

In [ ]:
def IsProdLink(link):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
        response = requests.get(link, headers=headers, timeout=6)
        soup = BeautifulSoup(response.content, "html.parser")

        # Only look for company-specific infoboxes
        infobox = soup.find("table", class_=lambda c: c and (
            "infobox ib-company vcard" in c or
            "infobox vcard plainlist ib-company" in c
        ))

        if not infobox:
            #print(f"[IsProdLink] No infobox found for: {link}")
            return False

        prod_identifiers = [
            "Company", "Company Type", "Formerly", "Industry", "Founded",
            "Founders", "Headquarters", "Products", "Parent"
        ]

        for th in infobox.find_all("th"):
            label = th.get_text(strip=True)
            if label in prod_identifiers:
                #print(f"[IsProdLink] Confirmed by label '{label}' → {link}")
                return True

        #print(f"[IsProdLink] Not a production company (no identifiers found): {link}")
        return False

    except Exception as e:
        print(f"[IsProdLink ERROR] {link} → {e}")
        return False


In [ ]:
print(IsProdLink("https://en.wikipedia.org/wiki/Universal_Pictures"))

True


## AcademyAwards Extraction Function

In [ ]:
#Extracting Academy Awards Info
def AcademyAwardsInfo(soup, Iteration):

  Table = soup.find("table" , {"class" : "infobox vevent"})
  TableBody = Table.find("tbody")
  #Intializing the Dictionary to store the information in
  Data = {}
  Data["Iteration"] = Iteration

  #Parsing the information card
  for tr in TableBody.find_all("tr"):
    th = tr.find("th")
    td = tr.find("td")
    if th and td:

      # Removing additional tags
      for sup in td.select("sup, span"):
        sup.extract()

      #Extracting Year
      if "Date" in th.text:
        Date = td.text.strip()
        Year = re.search(r'\d{4}', Date).group()
        Data["Year"] = Year

      #Extracting Site
      if "Site" in th.text:
        Site = td.text
        Data["Site"] = Site

      #Extracting the Information of the AcademyAwardsHost Table
      if "Hosted by" in th.text:
        a = td.find_all('a')
        for h in a:
          Host = h.text.strip()
          AcademyAwardsHost.append({
              "IterationNo" :Iteration,
              "Year": Year,
              "Hosted by": Host
          })


  AcademyAwards.append(Data)
  return Data





## Person Extraction Funtion

In [ ]:
#Building my PersonInformation funciton

def PersonInformation(link = None, Name = None, AllPersons = None):
# Parsing the person's website
  PersonInformation = {}

  #The following code will perform if a link was passed in the parameter
  if link:


    response = requests.get(link)
    soup = BeautifulSoup(response.content)
    IsDead = False


# Getting the person's name from the header
    FullName = soup.find("h1", {"id" : "firstHeading"})
    if FullName:
      CleanName = re.sub(r"\s*\(.*?\)\s*", "", FullName.text.strip())
      Name = CleanName.split()
      if len(Name) == 1:
        FName = Name[0]
        PersonInformation['FName'] = FName
        PersonInformation['MName'] = None
        PersonInformation['LName'] = "Unknown"
      elif len(Name) == 2:
        FName = Name[0]
        LName = Name[1]
        PersonInformation['FName'] = FName
        PersonInformation['MName'] = None
        PersonInformation['LName'] = LName

      elif len(Name) >=3:
        FName = Name[0]
        MName = Name[1]
        LName = Name[2]
        PersonInformation['FName'] = FName
        PersonInformation['MName'] = MName
        PersonInformation['LName'] = LName

# Parsing the information card
    Table = soup.find("table" , {"class": re.compile(r"infobox.*vcard|infobox.*plainlist")})
    if Table:
      TableBody = Table.find("tbody")

      for tr in TableBody.find_all("tr"):
          th = tr.find("th")
          td = tr.find("td")
          if th and td:
            # Removing additional tags
            for sup in td.select("sup, span"):
              sup.extract()

            #Extracting BirthDate and Country of Birth
            if "Born" in th.text:
              BirthDate = td.text.strip()
              BDate = ExtractDate(BirthDate)
              PersonInformation["BirthDate"] = BDate
              BirthPlace = td.find("div", {"class": "birthplace"})
              if BirthPlace:
                a = BirthPlace.find("a")
                if a:
                  CountryOfBirth = a.text.strip()
                  PersonInformation["CountryOfBirth"] = CountryOfBirth
                elif not a:
                  CountryOfBirth = BirthPlace.text.strip()
                  PersonInformation["CountryOfBirth"] = CountryOfBirth
                else:
                  PersonInformation["CountryOfBirth"] = 'Unknown'

            #Extracting DeathDate
            if "Died" in th.text:
              IsDead = True
              DeathDate = td.text.strip()
              DDate = ExtractDate(DeathDate)
              PersonInformation["DeathDate"] = DDate

          else:
            continue # There was an error as in the first row tr it had th only and not td so the code inside the if condition wouldn't excute

          if IsDead == False:
            PersonInformation["DeathDate"] = None

#The following code will perform if Name was passed in the parameter
  elif Name:
    PName = Name.strip().split()
    if len(PName) == 2:
      FName = PName[0]
      LName = PName[1]
      PersonInformation['FName'] = FName
      PersonInformation['MName'] = None
      PersonInformation['LName'] = LName
    elif len(PName) == 3:
      FName = PName[0]
      MName = PName[1]
      LName = PName[2]
      PersonInformation['FName'] = FName
      PersonInformation['MName'] = MName
      PersonInformation['LName'] = LName
    else:
      return None

    PersonInformation["BirthDate"] = None
    PersonInformation["CountryOfBirth"] = None
    PersonInformation["DeathDate"] = None


  if PersonInformation.get("FName"):
    AllPersons.append(PersonInformation)

  return PersonInformation

## Movie Extraction Function


In [ ]:
def MovieInformation(link):
  ProdComp = []
  Movie = {}
  ReleaseDt = ""
  t = ""

  # Parsing the web pages
  response = requests.get(link)
  soup = BeautifulSoup(response.content)
  Table = soup.find("table" , {"class" : "infobox vevent"})
  TableBody = Table.find("tbody")

  Title = soup.find("th" , {"class": "infobox-above summary"})
  if Title:
    t = Title.text.strip()
    Movie["Title"] = t
  else:
    t = None

  for tr in TableBody.find_all("tr"):
    #Parsing the title

    # Getting all the other information to store in  my movie table
    th = tr.find("th")
    td = tr.find("td")
    if th and td:

      #Removing additional tags
      for sup in td.select("sup, span"):
        sup.extract()

      #Parsing the Release date
      if  "Release date" in th.text:
        ul = td.find("ul")
        if ul:
          li = ul.find("li")
          if li:
            Date = li.text.strip().replace('\xa0', ' ')
            FullDate = ExtractDate(Date) # Had to extracrt yyyy-mm-dd in order to matche the date data type of mysql
            Movie["Release Date"] = FullDate
            ReleaseDt = FullDate
        else:
          Date = td.text.strip().replace('\xa0', ' ')
          FullDate = ExtractDate(Date) # Had to extracrt yyyy-mm-dd in order to matche the date data type of mysql
          Movie["Release Date"] = FullDate
          ReleaseDt = FullDate

      #Extracting Country
      if "Country" in th.text or "Countries" in th.text:
        ul = td.find("ul")
        if ul:
          li = ul.find("li")
          if li:
            Country = li.text.strip()
            Movie["Country"] = Country
        else:
          Country = td.text.strip()
          Movie["Country"] = Country

      #Extracting Language
      if "Language" in th.text:
        Language = td.text.strip()
        Movie["Language"] = Language

      #Getting the Running time of the movie
      if "Running time" in th.text:
        time = re.search(r'\d+', td.text)
        if time:
          Movie['Runtime'] = int(time.group())
        else:
          Movie['Runtime'] = None

      #Acquiring the production company or companies information for my MovieProductionCompany table
      if re.search(r'Production.*company|Distributed by|Production.*companies', th.text):
        ul = td.find("ul")
        if ul:
          for li in ul.find_all("li"):
            a = li.find("a")
            if a:
              Comp = a.text.strip()
              ProdComp.append(Comp)
        else:
          for a in td.find_all("a"):
            Comp = a.text.strip()
            ProdComp.append(Comp)

  for tr in TableBody.find_all("tr"):
    th = tr.find("th")
    td = tr.find("td")
    if th and td:
      #Removing additional tags
      for sup in td.select("sup, span"):
        sup.extract()
      # Acquiring the Roles and placing the information in my Persons Table
      for key, value in Roles.items():
        if key in th.text:
          if td.find_all("ul"):
            for li in td.find_all("li"):
              links = li.find_all("a")
              if links:
                for a in links:
                  href = a.get("href")
                  if href:
                    if href.startswith("http"):
                      link = href
                    else:
                      link = "https://en.wikipedia.org" + href
                  Person = PersonInformation(link=link, AllPersons=AllPersons)
                  if Person:
                    WorksAt.append({
                      "FName": Person["FName"],
                      "MName": Person["MName"],
                      "LName": Person["LName"],
                      "Title": t,
                      "ReleaseDate": ReleaseDt,
                      "Role": value
                  })
              else:
                name = li.text.strip()
                Person = PersonInformation(Name=name, AllPersons=AllPersons)
                if Person:
                  WorksAt.append({
                      "FName": Person["FName"],
                      "MName": Person["MName"],
                      "LName": Person["LName"],
                      "Title": t,
                      "ReleaseDate": ReleaseDt,
                      "Role": value
                  })
          else:
            links = td.find_all("a")
            if links:
                for a in links:
                  href = a.get("href")
                  if href:
                    if href.startswith("http"):
                      link = href
                    else:
                      link = "https://en.wikipedia.org" + href
                  Person = PersonInformation(link=link, AllPersons=AllPersons)
                  if Person:
                    WorksAt.append({
                      "FName": Person["FName"],
                      "MName": Person["MName"],
                      "LName": Person["LName"],
                      "Title": t,
                      "ReleaseDate": ReleaseDt,
                      "Role": value
                    })
            else:
              name = td.text.strip()
              Person = PersonInformation(Name=name, AllPersons=AllPersons)
              if Person:
                WorksAt.append({
                      "FName": Person['FName'],
                      "MName": Person["MName"],
                      "LName": Person["LName"],
                      "Title": t,
                      "ReleaseDate": ReleaseDt,
                      "Role": value
                  })

  for c in ProdComp:
    MovieProdComp.append({
            "Title": t,
            "ReleaseDate": ReleaseDt,
            "ProductionCompany": c
            })


  Movies.append(Movie)

  return {"Movie": Movie}

## Nomination Extraction Function

In [ ]:
def NominationInformation(soup, IterationNo, Year):
  Table = soup.find("table", {"class": "wikitable"})
  TableBody = Table.find("tbody")
  Category = " "


  for tr in TableBody.find_all("tr"):
    for td in tr.find_all("td"):
      d = td.find("div")
      if d:
        a = d.find("a")
        if a:
          Category = a.text.strip()
        else:
          Category = d.text.strip()
      for OuterUl in td.find_all("ul", recursive = False):
        for li in OuterUl.find_all("li", recursive = False):
          ExtractNomination(li, Category, IterationNo, Year, IsGranted = True)

          InnerUl = li.find("ul")
          if InnerUl:
            for NomineeLi in InnerUl.find_all("li", recursive = False):
                ExtractNomination(NomineeLi, Category, IterationNo, Year, IsGranted = False)



def ExtractNomination(li, Category, IterationNo, Year, IsGranted):

  Nom = {
      "IterationNo": IterationNo,
      "Year": Year,
      "Category": Category,
      "IsGranted": IsGranted,
      "MovieTitle": None,
      "ReleaseDate": None,
      "FName": None,
      "MName": None,
      "LName": None
  }

  MovieFound = False
  PersonFound = False

  Links = li.find_all("a")
  for a in Links:
    href = a.get("href")
    if href:
      if href.startswith("http"):
          Link = href
      else:
          Link = "https://en.wikipedia.org" + href
    if IsProdLink(Link):
      continue
    if not MovieFound and IsLinkMovie(Link):
      Movie = MovieInformation(Link)
      if Movie:
        Nom["MovieTitle"] = Movie["Movie"].get("Title")
        Nom["ReleaseDate"] = Movie["Movie"].get("Release Date")
        MovieFound = True
    elif not PersonFound and IsLinkPerson(Link):
      Person = PersonInformation(Link, AllPersons = AllPersons)
      if Person:
          Nom["FName"] = Person["FName"]
          Nom["MName"] = Person["MName"]
          Nom["LName"] = Person["LName"]
          PersonFound = True

  if not Links:
    Name = li.text.strip()
    if Name:
      P = PersonInformation(Name = Name , AllPersons = AllPersons)
      if P:
        Nom["FName"] = P["FName"]
        Nom["MName"] = P["MName"]
        Nom["LName"] = P["LName"]

  Nomination.append(Nom)

In [ ]:
i = 1
SuffixNum = '1st'
URL = f"https://en.wikipedia.org/wiki/{SuffixNum}_Academy_Awards"
print(f"{SuffixNum} Academy Awards")
try:
  response = requests.get(URL)
  soup = BeautifulSoup(response.content)
  A = AcademyAwardsInfo(soup, i)
  NominationInformation(soup, A["Iteration"], A["Year"])
  time.sleep(1.5)

except Exception as E:
  print(f"Failed To Extract Iteration {i}: {E}")
  time.sleep(5)

1st Academy Awards


# Webscraping the 96 iterations

In [ ]:
for i in range(1,97):
  SuffixNum = number.ordinal(i)
  URL = f"https://en.wikipedia.org/wiki/{SuffixNum}_Academy_Awards"
  print(f"{SuffixNum} Academy Awards")
  try:
    session = requests.Session()
    session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    })
    response = session.get(URL, timeout=10)
    soup = BeautifulSoup(response.content)
    A = AcademyAwardsInfo(soup, i)
    NominationInformation(soup, A["Iteration"], A["Year"])
    time.sleep(1.25)

  except Exception as E:
    print(f"Failed To Extract Iteration {i}: {E}")
    time.sleep(5)

1st Academy Awards
2nd Academy Awards
3rd Academy Awards
4th Academy Awards
5th Academy Awards
6th Academy Awards
7th Academy Awards
8th Academy Awards
9th Academy Awards
10th Academy Awards
11th Academy Awards
12th Academy Awards
13th Academy Awards
14th Academy Awards
15th Academy Awards
16th Academy Awards
17th Academy Awards
18th Academy Awards
19th Academy Awards
20th Academy Awards
21st Academy Awards
22nd Academy Awards
23rd Academy Awards
24th Academy Awards
25th Academy Awards
26th Academy Awards
27th Academy Awards
28th Academy Awards
29th Academy Awards
30th Academy Awards
31st Academy Awards
32nd Academy Awards
33rd Academy Awards


## Storing Values Into CSV

In [ ]:
PersonDf = pd.DataFrame(AllPersons)
PersonDf.to_csv("Person.csv", index = False)

WorksAtDf = pd.DataFrame(WorksAt)
WorksAtDf.to_csv("WorksAt.csv", index = False)

MovieDf = pd.DataFrame(Movies)
MovieDf.to_csv("Movie.csv", index = False)

MovieProdCompDf = pd.DataFrame(MovieProdComp)
MovieProdCompDf.to_csv("MovieProdComp.csv", index = False)

AcademyAwardsDf = pd.DataFrame(AcademyAwards)
AcademyAwardsDf.to_csv("AcademyAwards.csv", index = False)

AcademyAwardsHostDf = pd.DataFrame(AcademyAwardsHost)
AcademyAwardsHostDf.to_csv("AcademyAwardsHost.csv", index = False)

NominationDf = pd.DataFrame(Nomination)
NominationDf.to_csv("Nomination.csv", index = False)

# Sample Test of User

In [ ]:
!pip install faker
from faker import Faker
import random
from datetime import datetime

Fake = Faker()

def GenerateUsers(n):
  User = []
  for i in range(1, (n+1)):
    EmailAddress = Fake.email()
    UserName = Fake.user_name()
    BirthDate = Fake.date_of_birth(minimum_age = 18, maximum_age = 80)
    Age = datetime.now().year - BirthDate.year
    Gender = random.choice(['M' , 'F'])
    CountryOfBirth = Fake.country()

    User.append({
        "EmailAddress": EmailAddress,
        "UserName": UserName,
        "BirthDate" : BirthDate ,
        "Age" : Age ,
        "Gender" : Gender ,
        "CountryOfBirth" : CountryOfBirth
    })

    i = i + 1

  return User


UserRandom = GenerateUsers(10)
print(UserRandom)

[{'EmailAddress': 'thogan@example.com', 'UserName': 'tmartinez', 'BirthDate': datetime.date(1975, 7, 17), 'Age': 50, 'Gender': 'M', 'CountryOfBirth': 'British Indian Ocean Territory (Chagos Archipelago)'}, {'EmailAddress': 'hdurham@example.org', 'UserName': 'brittneyweber', 'BirthDate': datetime.date(1963, 9, 13), 'Age': 62, 'Gender': 'M', 'CountryOfBirth': 'Tonga'}, {'EmailAddress': 'johnpeterson@example.org', 'UserName': 'craig10', 'BirthDate': datetime.date(1996, 7, 2), 'Age': 29, 'Gender': 'M', 'CountryOfBirth': 'Bouvet Island (Bouvetoya)'}, {'EmailAddress': 'frank40@example.org', 'UserName': 'traceywhitney', 'BirthDate': datetime.date(1965, 12, 28), 'Age': 60, 'Gender': 'F', 'CountryOfBirth': 'Mauritius'}, {'EmailAddress': 'zrodriguez@example.org', 'UserName': 'zbrown', 'BirthDate': datetime.date(1946, 9, 18), 'Age': 79, 'Gender': 'M', 'CountryOfBirth': 'Guinea'}, {'EmailAddress': 'stricklanddaniel@example.com', 'UserName': 'olivingston', 'BirthDate': datetime.date(2007, 3, 20), '

In [ ]:
def GenerateUserNominations(n, Users):
  Categories = ["Best Actor", "Best Cinematography", "Best Writing", "Best Picture", "Best Actress", "Best Director", "Best Production"]
  UserNom = []
  for i in range(1, (n+1)):
    User = random.choice(Users)
    Year = random.randint(1930,2024)
    Iteration = random.randint(1,97)
    MovieTitle = random.choice(Movies)["Title"]
    ReleaseDate = random.choice(Movies)["Release Date"]
    FName = random.choice(AllPersons)["FName"]
    MName = random.choice(AllPersons)["MName"]
    LName = random.choice(AllPersons)["LName"]
    Category = random.choice(Categories)

    UserNom.append({
        "EmailAddress": User["EmailAddress"],
        "IterationNo": Iteration,
        "Year": Year,
        "Category": Category,
        "MovieTitle": MovieTitle,
        "ReleaseDate": ReleaseDate,
        "FName": FName,
        "MName": MName,
        "LName": LName
    })

    i = i + 1

  return UserNom


UserNomRandom = GenerateUserNominations(10, UserRandom)
print(UserNomRandom)




[{'EmailAddress': 'stricklanddaniel@example.com', 'IterationNo': 72, 'Year': 1958, 'Category': 'Best Cinematography', 'MovieTitle': 'The Crowd', 'ReleaseDate': '1927-05-06', 'FName': 'George', 'MName': None, 'LName': 'Robinson'}, {'EmailAddress': 'brookstroy@example.com', 'IterationNo': 75, 'Year': 1987, 'Category': 'Best Actress', 'MovieTitle': 'Speedy', 'ReleaseDate': '1928-01-07', 'FName': 'Otto', 'MName': 'Hermann', 'LName': 'Arlen'}, {'EmailAddress': 'brookstroy@example.com', 'IterationNo': 87, 'Year': 1976, 'Category': 'Best Cinematography', 'MovieTitle': 'The Noose', 'ReleaseDate': '1928-01-29', 'FName': 'Samuel', 'MName': None, 'LName': 'Korda'}, {'EmailAddress': 'zrodriguez@example.org', 'IterationNo': 62, 'Year': 1939, 'Category': 'Best Production', 'MovieTitle': 'Sunrise: A Song of Two Humans', 'ReleaseDate': '1927-11-04', 'FName': 'Herbert', 'MName': None, 'LName': 'Chaplin'}, {'EmailAddress': 'stricklanddaniel@example.com', 'IterationNo': 12, 'Year': 1952, 'Category': 'Bes

In [ ]:
UserDf = pd.DataFrame(UserRandom)
UserDf.to_csv("User.csv", index = False)

UserNomDf = pd.DataFrame(UserNomRandom)
UserNomDf.to_csv("UserNomination.csv", index = False)

        00:37:35	Drop table Oscars.User	Error Code: 3730. Cannot drop table 'user' referenced by a foreign key constraint 'usernomination_ibfk_3' on table 'usernomination'.	0.000 sec

        
